In [ ]:
"""
U-Net Training Script

Requirements:
    Python >= 3.9
    pip install numpy opencv-python tifffile matplotlib pillow scipy pandas scikit-learn tensorflow

Data folders:
    IMAGE_DIR -> frame_00000_ch1.tif, frame_00000_ch2.tif, ...
    MASK_DIR  -> frame_00000.tif  (labels encoded as 0/85/170/255)

Conventions used:
    - PEP 8 style: snake_case for functions/variables, UPPER_CASE for constants.
    - Clear top-of-file CONFIG so users can easily change key parameters/paths.
    - Short, precise English docstrings for every function/class/method:
      Purpose, Inputs, Outputs/Returns, and behavior notes where relevant.
"""

# =========================
# USER CONFIG (EDIT HERE)
# =========================
SAVE_DIR   = "./output_train"                   # Single folder for all outputs (weights, logs, optional full models)
IMAGE_DIR  = "./training_data/Full_DataSet/data"
MASK_DIR   = "./training_data/Full_DataSet/label"

SIZE         = 256
IN_CHANNELS  = 2
NUM_CLASSES  = 4
BATCH_SIZE   = 8
EPOCHS       = 100
LEARNING_RATE = 1e-3

VAL_SPLIT     = 0.1
RANDOM_STATE  = 0

CLASS_WEIGHTS = [4, 2, 2, 1]

# Resume control:
RESUME_TRAINING = True        # True: load model_last.weights.h5 + continue from CSV; False: start fresh

# Saving cadence:
SAVE_WEIGHTS_EVERY_N = 1      # Save weights every N epochs (periodic snapshots)
SAVE_FULL_MODEL_EVERY_N = None  # None to disable; or e.g. 10 to also save a full model every 10 epochs

# =========================
# DERIVED PATHS
# =========================
import os
os.makedirs(SAVE_DIR, exist_ok=True)

WEIGHTS_PATH         = os.path.join(SAVE_DIR, "model_last.weights.h5")
BEST_WEIGHTS_PATH    = os.path.join(SAVE_DIR, "model_best.weights.h5")
LOG_PATH             = os.path.join(SAVE_DIR, "training_log.csv")
FULL_MODEL_LAST_PATH = os.path.join(SAVE_DIR, "model_last.h5")
FULL_MODEL_BEST_PATH = os.path.join(SAVE_DIR, "model_best.h5")

# =========================
# IMPORTS
# =========================
import cv2
import numpy as np
import pandas as pd
import random
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation, MaxPool2D,
    Conv2DTranspose, Concatenate, Input
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, Callback
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split

# =========================
# LOSS
# =========================
def weighted_categorical_crossentropy(weights):
    """
    Weighted categorical crossentropy loss.

    Purpose:
        Apply class-dependent weights to the standard categorical crossentropy.

    Inputs:
        weights (list[float]): Class weights of length NUM_CLASSES.

    Returns:
        loss (callable): Keras-compatible loss function:
            Args:
                y_true: one-hot mask, shape (B, H, W, NUM_CLASSES)
                y_pred: softmax probabilities, shape (B, H, W, NUM_CLASSES)
            Returns:
                Tensor of shape (B, H, W) with per-pixel loss values.
    """
    weights = K.variable(weights)
    def loss(y_true, y_pred):
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss_val = y_true * K.log(y_pred) * weights
        loss_val = -K.sum(loss_val, -1)
        return loss_val
    return loss

loss_fn = weighted_categorical_crossentropy(CLASS_WEIGHTS)

# =========================
# MODEL (U-Net)
# =========================
def conv_block(inputs, num_filters):
    """
    Two Conv2D + BatchNorm + ReLU layers.

    Inputs:
        inputs: input tensor.
        num_filters (int): number of convolution filters.

    Returns:
        Tensor after two conv-BN-ReLU operations with 'same' padding.
    """
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

def encoder_block(inputs, num_filters):
    """
    Encoder stage: conv_block + 2x2 MaxPool.

    Inputs:
        inputs: input tensor.
        num_filters (int): filters for conv_block.

    Returns:
        skip (Tensor): features before pooling (for decoder skip connection).
        pooled (Tensor): downsampled features (MaxPool).
    """
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2,2))(x)
    return x, p

def decoder_block(inputs, skip, num_filters):
    """
    Decoder stage: up-convolution + concatenate skip + conv_block.

    Inputs:
        inputs: decoder input tensor.
        skip: corresponding encoder skip tensor.
        num_filters (int): filters for conv_block.

    Returns:
        Tensor after upsampling, concatenation, and conv_block.
    """
    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    """
    Build the U-Net architecture used for multi-class segmentation.

    Inputs:
        input_shape (tuple): (H, W, C) e.g., (SIZE, SIZE, IN_CHANNELS)

    Returns:
        model (tf.keras.Model): U-Net with softmax output of NUM_CLASSES channels.
    """
    inputs = Input(input_shape)
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    b1 = conv_block(p4, 1024)
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    outputs = Conv2D(NUM_CLASSES, 1, padding="same", activation="softmax")(d4)
    model = Model(inputs, outputs, name="UNET")
    return model

# =========================
# DATA GENERATOR
# =========================
class TiffDataGenerator(Sequence):
    """
    Keras Sequence that streams paired channels and segmentation masks from disk.

    Purpose:
        - Load grayscale channels ch1/ch2 and a mask per sample.
        - Apply CLAHE + Gaussian blur + percentile normalization (1–99).
        - Map LUT mask values {0, 85, 170, 255} to class indices {0, 1, 2, 3}.
        - Yield (images, masks) batches for training/validation.

    Init Inputs:
        image_dir (str): folder of ch1/ch2 TIFFs (e.g., frame_XXXXX_ch1.tif / _ch2.tif).
        mask_dir  (str): folder of label TIFFs (frame_XXXXX.tif).
        filenames (list[str]): base names (without suffixes) to load.
        batch_size (int): batch size.
        shuffle (bool): whether to shuffle after each epoch.

    Yields:
        images (np.ndarray): float32, shape (B, SIZE, SIZE, 2).
        masks  (np.ndarray): one-hot uint8->float, shape (B, SIZE, SIZE, NUM_CLASSES).
    """
    def __init__(self, image_dir, mask_dir, filenames, batch_size=8, shuffle=True):
        super().__init__()
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.filenames = filenames
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.SIZE = SIZE
        self.NUM_CLASSES = NUM_CLASSES
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        self.on_epoch_end()

    def __len__(self):
        """
        Returns:
            int: number of batches per epoch (ceil(N / batch_size)).
        """
        return int(np.ceil(len(self.filenames) / self.batch_size))

    def on_epoch_end(self):
        """Shuffle file order at the end of each epoch if shuffle=True."""
        if self.shuffle:
            random.shuffle(self.filenames)

    def normalize_channel(self, image):
        """
        Percentile-based normalization after CLAHE/blur.

        Inputs:
            image (np.ndarray): grayscale channel.

        Returns:
            np.ndarray: normalized channel in [0, 1].
        """
        per99 = np.percentile(image, 99)
        per1  = np.percentile(image, 1)
        image = np.clip(image, per1, per99)
        return (image - per1) / (per99 - per1 + 1e-8)

    def __getitem__(self, index):
        """
        Build one batch.

        Inputs:
            index (int): batch index.

        Returns:
            images (np.ndarray): (B, SIZE, SIZE, 2), float32.
            masks  (np.ndarray): (B, SIZE, SIZE, NUM_CLASSES), one-hot.
        """
        batch_filenames = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        images, masks = [], []

        for base in batch_filenames:
            ch1_path = os.path.join(self.image_dir, f"{base}_ch1.tif")
            ch2_path = os.path.join(self.image_dir, f"{base}_ch2.tif")
            mask_path = os.path.join(self.mask_dir,   f"{base}.tif")

            ch1 = cv2.imread(ch1_path, cv2.IMREAD_GRAYSCALE)
            ch2 = cv2.imread(ch2_path, cv2.IMREAD_GRAYSCALE)
            msk = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if ch1 is None or ch2 is None or msk is None:
                continue

            ch1 = cv2.resize(ch1, (self.SIZE, self.SIZE), interpolation=cv2.INTER_AREA)
            ch2 = cv2.resize(ch2, (self.SIZE, self.SIZE), interpolation=cv2.INTER_AREA)
            msk = cv2.resize(msk, (self.SIZE, self.SIZE), interpolation=cv2.INTER_NEAREST)

            ch1 = self.normalize_channel(self.clahe.apply(cv2.GaussianBlur(ch1, (3,3), 0)))
            ch2 = self.normalize_channel(self.clahe.apply(cv2.GaussianBlur(ch2, (3,3), 0)))
            img = np.stack([ch1, ch2], axis=-1).astype(np.float32)

            msk = msk.copy()
            msk[msk == 85]  = 1
            msk[msk == 170] = 2
            msk[msk == 255] = 3

            images.append(img)
            masks.append(msk)

        images = np.array(images, dtype=np.float32)
        masks  = np.array(masks,  dtype=np.uint8)
        masks  = to_categorical(masks, num_classes=self.NUM_CLASSES)
        return images, masks

# =========================
# TRAIN/VAL SPLIT
# =========================
# Purpose:
#   Collect base names from MASK_DIR and split into train/val by VAL_SPLIT and RANDOM_STATE.
all_filenames = sorted([f.replace(".tif", "") for f in os.listdir(MASK_DIR) if f.endswith(".tif")])
train_filenames, val_filenames = train_test_split(
    all_filenames, test_size=VAL_SPLIT, random_state=RANDOM_STATE
)
train_gen = TiffDataGenerator(IMAGE_DIR, MASK_DIR, train_filenames, batch_size=BATCH_SIZE)
val_gen   = TiffDataGenerator(IMAGE_DIR, MASK_DIR, val_filenames,   batch_size=BATCH_SIZE, shuffle=False)

# =========================
# MODEL + COMPILE + RESUME
# =========================
# Purpose:
#   Build the network, compile with the weighted loss, and optionally resume training
#   from the last saved epoch/weights using CSV length and model_last.weights.h5.
model = build_unet((SIZE, SIZE, IN_CHANNELS))
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss=loss_fn,
              metrics=["accuracy"],
              run_eagerly=True)

initial_epoch = 0
if RESUME_TRAINING:
    if os.path.exists(LOG_PATH):
        try:
            log_df = pd.read_csv(LOG_PATH)
            initial_epoch = len(log_df)
        except Exception:
            initial_epoch = 0
    if os.path.exists(WEIGHTS_PATH):
        print("Loading weights from previous checkpoint...")
        model.load_weights(WEIGHTS_PATH)

# =========================
# CALLBACKS
# =========================
class PeriodicSaver(Callback):
    """
    Save weights every N epochs and (optionally) a full model at a fixed cadence.

    Init Inputs:
        save_every (int): frequency (in epochs) for saving weight snapshots.
        save_dir (str): target folder for the snapshots.
        prefix (str): filename prefix for weight snapshots.

    Behavior:
        - Always saves weights every `save_every` epochs as <prefix>_<epoch>.weights.h5
        - If SAVE_FULL_MODEL_EVERY_N is set (int), also saves model_epoch_<epoch>.h5 periodically.
    """
    def __init__(self, save_every, save_dir, prefix="weights_epoch"):
        super().__init__()
        self.save_every = save_every
        self.save_dir = save_dir
        self.prefix = prefix

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_every == 0:
            w_path = os.path.join(self.save_dir, f"{self.prefix}_{epoch+1}.weights.h5")
            self.model.save_weights(w_path)
            print(f"\nSaved weights at epoch {epoch+1} -> {w_path}")
        if SAVE_FULL_MODEL_EVERY_N and (epoch + 1) % SAVE_FULL_MODEL_EVERY_N == 0:
            f_path = os.path.join(self.save_dir, f"model_epoch_{epoch+1}.h5")
            self.model.save(f_path)
            print(f"Saved full model at epoch {epoch+1} -> {f_path}")

checkpoint_last = ModelCheckpoint(
    WEIGHTS_PATH,
    save_best_only=False,
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
)
checkpoint_best = ModelCheckpoint(
    BEST_WEIGHTS_PATH,
    save_best_only=True,
    save_weights_only=True,
    monitor="val_loss",
    mode="min",
)

class SaveFullBest(Callback):
    """
    Save a full model file when a new best validation loss is reached.

    Init Inputs:
        best_path (str): output path for the best full model file.

    Behavior:
        - Active only if SAVE_FULL_MODEL_EVERY_N is set (not None).
        - Tracks best val_loss and saves FULL_MODEL_BEST_PATH when improved.
    """
    def __init__(self, best_path):
        super().__init__()
        self.best = np.inf
        self.best_path = best_path

    def on_epoch_end(self, epoch, logs=None):
        if not SAVE_FULL_MODEL_EVERY_N:
            return
        val_loss = logs.get("val_loss", None)
        if val_loss is not None and val_loss < self.best:
            self.best = val_loss
            self.model.save(self.best_path)
            print(f"Saved FULL MODEL (best) -> {self.best_path}")

full_best_cb = SaveFullBest(FULL_MODEL_BEST_PATH) if SAVE_FULL_MODEL_EVERY_N else None
csv_logger = CSVLogger(LOG_PATH, append=True)
periodic_saver = PeriodicSaver(save_every=SAVE_WEIGHTS_EVERY_N, save_dir=SAVE_DIR)

callbacks = [checkpoint_last, checkpoint_best, csv_logger, periodic_saver]
if full_best_cb:
    callbacks.append(full_best_cb)

# =========================
# TRAIN
# =========================
# Purpose:
#   Run model.fit with (optional) resume support via initial_epoch and preloaded weights.
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    initial_epoch=initial_epoch,
    callbacks=callbacks,
    verbose=1
)

# Optional: save a final full model at the end (enabled only if SAVE_FULL_MODEL_EVERY_N is set)
if SAVE_FULL_MODEL_EVERY_N:
    model.save(FULL_MODEL_LAST_PATH)
